In [ ]:
import pickle
import pandas as pd
import numpy as np
import customtkinter
from IPython.display import display
from customtkinter import filedialog

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def preprocess(data):
    new_data = []

    ### manual encoding
    for idx in range(18):    
        # encoding the contract feature
        if idx == 2 :
            val = data[idx]
            if val == "Month-to-month" :
                new_data.append(0)
            elif val == "One year" :
                new_data.append(1)
            else :
                new_data.append(2)
                
        # encoding the gender feature
        elif idx == 5 :
            val = data[idx]
            if val == "Female" :
                new_data.append(0)
            else :
                new_data.append(1)

        # encoding the InternetService feature
        elif idx == 10 :
            val = data[idx]
            if val == "DSL" :
                new_data.append(0)
                new_data.append(0)
            elif val == "Fiber optic" :
                new_data.append(1)
                new_data.append(0)
            else :
                new_data.append(0)
                new_data.append(1)

        # encoding the PaymentMethod feature
        elif idx == 17 :
            val = data[idx]
            if val == "Bank transfer (automatic)" :
                new_data.append(0)
                new_data.append(0)
                new_data.append(0)
            elif val == "Credit card (automatic)" :
                new_data.append(1)
                new_data.append(0)
                new_data.append(0)
            elif val == "Electronic check" :
                new_data.append(0)
                new_data.append(1)
                new_data.append(0)
            else :
                new_data.append(0)
                new_data.append(0)
                new_data.append(1)

        else :
            new_data.append(data[idx])
            
    # scaling numerical values
    with open("scaler.pkl","rb") as pickle_in2 :
        scaler = pickle.load(pickle_in2)
    
    vals = [new_data[i] for i in [1,3,4]]    
    scaled = scaler.transform([vals])[0].tolist()
    
    new_data[1] = scaled[0]
    new_data[3] = scaled[1]
    new_data[4] = scaled[2]
    
    return new_data

In [ ]:
def predictOne(data):
    new_data = preprocess(data)
    
    with open("model.pkl","rb") as pickle_in :
        model = pickle.load(pickle_in)
        
    with open("selector.pkl","rb") as pickle_in3 :
        selector = pickle.load(pickle_in3)
    
    pred = model.predict(selector.transform([new_data]))
    
    if pred[0] == 0:
        return("No (Will Not Churn)")
    else:
        return("Yes (Will Churn)")

In [ ]:
def predict(data):
    preds = []
    for row in data :
        output = predictOne(row)
        preds.append(output)
    return preds

In [ ]:
def predictDF(df):

    # step 1 removing unnecessary columns and modifying the data
    df = df.drop(["customerID","StreamingMovies"],axis=1)
    modify_cols = ["OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV"]
    for col in modify_cols:
        df[col] = df[col].replace({"No internet service":"No"})
    df["MultipleLines"] = df["MultipleLines"].replace({"No phone service":"No"})
    
    
    # step 2  encoding few features
    modify_cols2 = ["SeniorCitizen","tenure","MonthlyCharges","TotalCharges","gender","Partner","Dependents","PhoneService",\
                    "MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport",\
                    "StreamingTV","Contract","PaperlessBilling","PaymentMethod"]
    for col2 in modify_cols2:
        df[col2] = df[col2].replace({"No":0})
        df[col2] = df[col2].replace({"Yes":1})
        df[col2] = df[col2].replace([" ",np.NAN,np.NaN,np.nan],None)
    
    
    # step 3  dealing with null values
    modify_cols3 = ["SeniorCitizen","tenure","MonthlyCharges","TotalCharges","Partner","Dependents","PhoneService",\
                    "MultipleLines","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport",\
                    "StreamingTV","PaperlessBilling"]
    for col3 in modify_cols3:
        df[col3] = df[col3].replace({None:0})
    
    df["gender"] = df["gender"].replace({None:"Male"})
    df["InternetService"] = df["InternetService"].replace({None:"No"})
    df["Contract"] = df["Contract"].replace({None:"Month-to-month"})
    df["PaymentMethod"] = df["PaymentMethod"].replace({None:"Mailed check"})
    
    
    # step 4  make sure that the numeric features are actually numeric
    df["TotalCharges"] = df["TotalCharges"].astype(float)
    df["MonthlyCharges"] = df["MonthlyCharges"].astype(float)
    df["tenure"] = df["tenure"].astype(float)
   

    # step 5  re-ordering the data the same order that the model trained on 
    data = df[["SeniorCitizen","tenure","Contract","MonthlyCharges","TotalCharges","gender","Partner","Dependents","PhoneService",\
           "MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport",\
           "StreamingTV","PaperlessBilling","PaymentMethod"]]
    
    return (predict(np.array(data)))

# 
# 
# 

## If You Want To Enter Few Rows Manually Run The Cell Below
#### Note : all values must be entered For Accurate Prediction. ( 0 ==> No , 1 ==> Yes )

In [ ]:
customtkinter.set_appearance_mode("dark")
customtkinter.set_default_color_theme("green")

window = customtkinter.CTk()
window.title("Customer Data")
window.geometry("1200x500")

frame = customtkinter.CTkFrame(window)
frame.pack(fill="both",expand=True)

customer_data_frame = customtkinter.CTkLabel(frame,text="Enter The Customer Data",font=("Arial",20,"bold"))
customer_data_frame.grid(row=0,column=0,padx=50,pady=50)

#1
SeniorCitizen_label = customtkinter.CTkLabel(customer_data_frame, text="SeniorCitizen (0 or 1)")
SeniorCitizen_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["0", "1"])
SeniorCitizen_label.grid(row=2, column=0)
SeniorCitizen_combobox.grid(row=2, column=1)

#2
Tenure_label = customtkinter.CTkLabel(customer_data_frame, text="Tenure")
Tenure_entry = customtkinter.CTkEntry(customer_data_frame,width=145)
Tenure_label.grid(row=3, column=0)
Tenure_entry.grid(row=3, column=1)
Tenure_entry.insert(0, "0")

#3
MonthlyCharges_label = customtkinter.CTkLabel(customer_data_frame, text="MonthlyCharges")
MonthlyCharges_entry = customtkinter.CTkEntry(customer_data_frame,width=145)
MonthlyCharges_label.grid(row=4, column=0)
MonthlyCharges_entry.grid(row=4, column=1)
MonthlyCharges_entry.insert(0, "0")


#4
TotalCharges_label = customtkinter.CTkLabel(customer_data_frame, text="TotalCharges")
TotalCharges_entry = customtkinter.CTkEntry(customer_data_frame,width=145)
TotalCharges_label.grid(row=5, column=0)
TotalCharges_entry.grid(row=5, column=1)
TotalCharges_entry.insert(0, "0")


#5
Gender_label = customtkinter.CTkLabel(customer_data_frame, text="Gender")
Gender_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["Male", "Female"])
Gender_label.grid(row=6, column=0)
Gender_combobox.grid(row=6, column=1)

#6
Partner_label = customtkinter.CTkLabel(customer_data_frame, text="Partner (0 or 1)")
Partner_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["0", "1"])
Partner_label.grid(row=7, column=0)
Partner_combobox.grid(row=7, column=1)

#7
Dependents_label = customtkinter.CTkLabel(customer_data_frame, text="Dependents (0 or 1)")
Dependents_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["0", "1"])
Dependents_label.grid(row=8, column=0)
Dependents_combobox.grid(row=8, column=1)

#8
PhoneService_label = customtkinter.CTkLabel(customer_data_frame, text="PhoneService (0 or 1)")
PhoneService_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["0", "1"])
PhoneService_label.grid(row=9, column=0)
PhoneService_combobox.grid(row=9, column=1)

#9
MultipleLines_label = customtkinter.CTkLabel(customer_data_frame, text="MultipleLines (0 or 1)")
MultipleLines_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["0", "1"])
MultipleLines_label.grid(row=10, column=0)
MultipleLines_combobox.grid(row=10, column=1)

#10
InternetService_label = customtkinter.CTkLabel(customer_data_frame, text="InternetService")
InternetService_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["No","DSL","Fiber optic"])
InternetService_label.grid(row=2, column=4)
InternetService_combobox.grid(row=2, column=5)

#11
OnlineSecurity_label = customtkinter.CTkLabel(customer_data_frame, text="OnlineSecurity (0 or 1)")
OnlineSecurity_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["0", "1"])
OnlineSecurity_label.grid(row=3, column=4)
OnlineSecurity_combobox.grid(row=3, column=5)

#12
OnlineBackup_label = customtkinter.CTkLabel(customer_data_frame, text="OnlineBackup (0 or 1)")
OnlineBackup_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["0", "1"])
OnlineBackup_label.grid(row=4, column=4)
OnlineBackup_combobox.grid(row=4, column=5)


#13
DeviceProtection_label = customtkinter.CTkLabel(customer_data_frame, text="DeviceProtection (0 or 1)")
DeviceProtection_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["0", "1"])
DeviceProtection_label.grid(row=5, column=4)
DeviceProtection_combobox.grid(row=5, column=5)

#14
TechSupport_label = customtkinter.CTkLabel(customer_data_frame, text="TechSupport (0 or 1)")
TechSupport_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["0", "1"])
TechSupport_label.grid(row=6, column=4)
TechSupport_combobox.grid(row=6, column=5)

#15
StreamingTV_label = customtkinter.CTkLabel(customer_data_frame, text="StreamingTV (0 or 1)")
StreamingTV_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["0", "1"])
StreamingTV_label.grid(row=7, column=4)
StreamingTV_combobox.grid(row=7, column=5)

#16
Contract_label = customtkinter.CTkLabel(customer_data_frame, text="Contract")
Contract_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["Month-to-month","One year","Two year"])
Contract_label.grid(row=8, column=4)
Contract_combobox.grid(row=8, column=5)

#17
PaperlessBilling_label = customtkinter.CTkLabel(customer_data_frame, text="PaperlessBilling (0 or 1)")
PaperlessBilling_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["0", "1"])
PaperlessBilling_label.grid(row=9, column=4)
PaperlessBilling_combobox.grid(row=9, column=5)

#18
PaymentMethod_label = customtkinter.CTkLabel(customer_data_frame, text="PaymentMethod")
PaymentMethod_combobox = customtkinter.CTkComboBox(customer_data_frame,values=["Electronic check","Mailed check","Bank transfer (automatic)","Credit card (automatic)"])
PaymentMethod_label.grid(row=10, column=4)
PaymentMethod_combobox.grid(row=10, column=5)



###################################################################################################################
def onClick():
    SeniorCitizen = int(SeniorCitizen_combobox.get())
    tenure = eval(Tenure_entry.get())
    monthlyCharges = eval(MonthlyCharges_entry.get())    
    TotalCharges = eval(TotalCharges_entry.get())    
    gender = str(Gender_combobox.get())    
    partner = int(Partner_combobox.get())    
    dependents = int(Dependents_combobox.get())    
    phoneService = int(PhoneService_combobox.get())
    MultipleLines = int(MultipleLines_combobox.get())
    InternetService = str(InternetService_combobox.get())
    OnlineSecurity = int(OnlineSecurity_combobox.get())
    OnlineBackup = int(OnlineBackup_combobox.get())
    DeviceProtection = int(DeviceProtection_combobox.get())
    TechSupport = int(TechSupport_combobox.get())
    StreamingTV = int(StreamingTV_combobox.get())
    Contract = str(Contract_combobox.get())
    PaperlessBilling = int(PaperlessBilling_combobox.get())
    PaymentMethod = str(PaymentMethod_combobox.get())
    
    data = [[SeniorCitizen,tenure,Contract,monthlyCharges,TotalCharges,gender,partner,dependents,phoneService,MultipleLines,\
           InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,PaperlessBilling,\
           PaymentMethod]]
    
    output = predict(data)[0]

    prediction_label = customtkinter.CTkLabel(customer_data_frame, text="Prediction : ",font=("Arial",20,"bold"))
    prediction_label.grid(row=17, column=0)
    prediction.configure(text = output)
    
###################################################################################################################

button = customtkinter.CTkButton(customer_data_frame, text="Submit Data",command=onClick)
button.grid(row = 15, column = 3, sticky = "news", padx = 20, pady = 10 )


prediction = customtkinter.CTkLabel(customer_data_frame,text=" ")
prediction.grid(row=17, column=1)

for widget in customer_data_frame.winfo_children():
    widget.grid_configure(padx=10,pady=5)

    
window.mainloop()

# 


## If You Want To enter A CSV File That Has All The Features Run The Cell Below
### required features :
#### customerID, gender, SeniorCitizen, Partner, Dependents,tenure, PhoneService, MultipleLines, InternetService,OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport,StreamingTV, StreamingMovies, Contract, PaperlessBilling,PaymentMethod, MonthlyCharges, TotalCharges.
#### All Features Must Be There Or Error Will Occur.
#### Note : order of the features doesn't matter.

In [ ]:
# method 1
# Enter your csv file that has the customer data.
# df = pd.read_csv('filename.csv')

# pred = predictDF(df)
# df["prediction"] = pred
# df

In [ ]:
# method 2
# run this cell to make a frame pop where you can select the file

customtkinter.set_appearance_mode("dark")
customtkinter.set_default_color_theme("green")
window = customtkinter.CTk()
window.title("Select CSV File To Open")
window.geometry("300x200")
frame = customtkinter.CTkFrame(window)
frame.pack(fill="both",expand=True)

file_frame = customtkinter.CTkLabel(frame,text="Select file to predict")
file_frame.grid(row=0,column=1,padx=50,pady=50)
  
def open_file():
    file = filedialog.askopenfile(mode ='r', filetypes =[('csv Files', '*.csv')])
    if file is not None:
        content = pd.read_csv(file)
        pred = predictDF(content)
        content['Prediction'] = pred
        display(content)
        label = customtkinter.CTkLabel(file_frame,text="Prediction is printed successfully !")
        label.grid(row=3, column=0)
        
btn = customtkinter.CTkButton(file_frame, text ='Browse', command = lambda:open_file())
btn.grid(row=1,column=0)

window.mainloop()